In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [2]:
processed_text = pd.read_csv("processed_text2.csv")
semantic_features = pd.read_csv("semantic_features_1.csv")
readiblity_features = pd.read_csv("readiblity_features_1.csv")
syntax_features = pd.read_csv("syntax_features_1.csv")
structure_features = pd.read_csv("structure_features_1.csv")
sentiment_features = pd.read_csv("sentiment_features_1.csv")
# features1 = pd.concat((semantic_features,readiblity_features,syntax_features,structure_features,sentiment_features),axis=1)
features1 = semantic_features
features1 

,Unigram 1 Vector,Unigram 2 Vector,Unigram 3 Vector,Bigram 1 Vector,Bigram 2 Vector,Bigram 3 Vector,0,1,2,3,...,292,293,294,295,296,297,298,299,perplexity,burstiness
0,0.289589,0.257691,0.176713,0.205008,0.195993,0.110211,3.298943,-0.097496,-1.243800,-1.556005,...,-0.023423,-0.012208,-0.038857,-0.112345,0.019806,-0.191705,0.004935,0.066934,0.980322,0.739900
1,0.399424,0.392514,0.252854,0.335707,0.135143,0.135143,2.469555,-1.520183,-0.088674,0.674373,...,0.091620,-0.035757,-0.010935,-0.263400,-0.028937,-0.132776,-0.047855,0.156044,0.978353,0.908240
2,0.252125,0.232873,0.228065,0.206809,0.206809,0.162793,4.028986,-0.967327,0.731313,-0.517026,...,0.097285,-0.062655,0.001630,-0.190409,0.049439,-0.187739,-0.068626,0.112607,0.987425,1.326533
3,0.329140,0.289508,0.228673,0.204215,0.204215,0.198036,4.886558,-0.795363,0.031693,-0.326789,...,-0.014526,-0.011976,-0.045313,-0.242068,0.068205,-0.262480,-0.037385,0.049851,0.982989,0.916779
4,0.235062,0.210388,0.195337,0.191618,0.146601,0.119208,4.981124,2.585339,-1.832572,-0.134134,...,0.029745,-0.018474,0.010237,-0.135530,-0.044580,-0.173416,-0.070213,-0.020181,0.988003,1.103673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14849,0.310523,0.227686,0.225792,0.184633,0.181347,0.113806,3.314267,-0.165267,-0.481785,0.934081,...,0.077112,-0.017818,-0.054787,-0.282352,0.074834,-0.205487,-0.032204,0.038809,0.985836,0.928619
14850,0.373180,0.281161,0.232597,0.233027,0.143635,0.143635,4.096208,-0.744611,1.659554,1.050656,...,0.013705,-0.032283,0.030066,-0.207891,-0.004106,-0.211200,0.087371,0.043869,0.979710,0.880013
14851,0.365121,0.249708,0.214290,0.284063,0.142031,0.142031,3.266862,0.567319,-1.951568,-0.244876,...,-0.057295,0.040614,0.023285,-0.080400,0.021114,-0.228245,-0.088153,-0.049829,0.978360,0.892065
14852,0.316609,0.232321,0.195385,0.212689,0.178228,0.110880,4.552253,-1.226738,-2.390434,1.397195,...,-0.108729,0.061012,-0.028177,-0.174479,0.044159,-0.138924,-0.011211,0.023251,0.986049,0.876717


In [3]:
# Add the 'label' column from processed_text to the semantic_features dataframe
features1['label'] = processed_text['label']

# Now we have a new dataframe with all semantic features and the label
features1_detect = features1

# Display the first few rows of the new dataframe
features1_detect.shape
features1_detect

,Unigram 1 Vector,Unigram 2 Vector,Unigram 3 Vector,Bigram 1 Vector,Bigram 2 Vector,Bigram 3 Vector,0,1,2,3,...,293,294,295,296,297,298,299,perplexity,burstiness,label
0,0.289589,0.257691,0.176713,0.205008,0.195993,0.110211,3.298943,-0.097496,-1.243800,-1.556005,...,-0.012208,-0.038857,-0.112345,0.019806,-0.191705,0.004935,0.066934,0.980322,0.739900,0
1,0.399424,0.392514,0.252854,0.335707,0.135143,0.135143,2.469555,-1.520183,-0.088674,0.674373,...,-0.035757,-0.010935,-0.263400,-0.028937,-0.132776,-0.047855,0.156044,0.978353,0.908240,0
2,0.252125,0.232873,0.228065,0.206809,0.206809,0.162793,4.028986,-0.967327,0.731313,-0.517026,...,-0.062655,0.001630,-0.190409,0.049439,-0.187739,-0.068626,0.112607,0.987425,1.326533,0
3,0.329140,0.289508,0.228673,0.204215,0.204215,0.198036,4.886558,-0.795363,0.031693,-0.326789,...,-0.011976,-0.045313,-0.242068,0.068205,-0.262480,-0.037385,0.049851,0.982989,0.916779,0
4,0.235062,0.210388,0.195337,0.191618,0.146601,0.119208,4.981124,2.585339,-1.832572,-0.134134,...,-0.018474,0.010237,-0.135530,-0.044580,-0.173416,-0.070213,-0.020181,0.988003,1.103673,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14849,0.310523,0.227686,0.225792,0.184633,0.181347,0.113806,3.314267,-0.165267,-0.481785,0.934081,...,-0.017818,-0.054787,-0.282352,0.074834,-0.205487,-0.032204,0.038809,0.985836,0.928619,1
14850,0.373180,0.281161,0.232597,0.233027,0.143635,0.143635,4.096208,-0.744611,1.659554,1.050656,...,-0.032283,0.030066,-0.207891,-0.004106,-0.211200,0.087371,0.043869,0.979710,0.880013,1
14851,0.365121,0.249708,0.214290,0.284063,0.142031,0.142031,3.266862,0.567319,-1.951568,-0.244876,...,0.040614,0.023285,-0.080400,0.021114,-0.228245,-0.088153,-0.049829,0.978360,0.892065,1
14852,0.316609,0.232321,0.195385,0.212689,0.178228,0.110880,4.552253,-1.226738,-2.390434,1.397195,...,0.061012,-0.028177,-0.174479,0.044159,-0.138924,-0.011211,0.023251,0.986049,0.876717,1


In [4]:
# Splitting the data into features (X) and target (y)
x = features1_detect.drop(columns=['label'])
y = features1_detect['label']

In [5]:
# 資料標準化
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(x)  # X 是原始的特徵資料

In [6]:
# 創建 SVM 模型
model = SVC(random_state=42)

In [7]:
def specificity_score(y_true, y_pred):
    tn, fp, _, _ = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    return specificity

specificity_scorer = make_scorer(specificity_score, greater_is_better=True)
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'specificity': specificity_scorer
}

In [8]:
# 設定 SVM 的網格搜索參數
param_grid = {
    'C': [1, 2, 3],              # 正則化參數
    'kernel': ['rbf'],          # 核函數類型
    'gamma': [0.1, 0.2, 0.3]  # RBF 核的 gamma 參數
}

In [9]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, refit='recall', cv=10, n_jobs=-1)

# 假設X_normalized和y已經被定義且正確預處理
# 執行網格搜索
grid_result = grid.fit(X_normalized, y)



In [10]:
cv_results = pd.DataFrame(grid_result.cv_results_)
print(cv_results[['param_C','param_gamma','mean_test_recall']])

   param_C  param_gamma  mean_test_recall
0        1          0.1          0.701206
1        1          0.2          0.696252
2        1          0.3          0.686757
3        2          0.1          0.708088
4        2          0.2          0.699831
5        2          0.3          0.692949
6        3          0.1          0.709601
7        3          0.2          0.702445
8        3          0.3          0.698041


In [11]:
# 假設 grid_result 是你的 GridSearchCV 結果
# 獲取所有結果


# 按照得分的排名進行排序，rank_test_score 越小表示排名越高
top_n_results = 6

# 顯示前 6 名的參數組合以及五個指標分數
top_6_params = cv_results.sort_values(by='mean_test_recall',ascending=False).head(top_n_results)
# 印出結果
for index, row in top_6_params.iterrows():
    print(f"參數組合: {row['params']}")
    print(f"Accuracy: {row['mean_test_accuracy']}")
    print(f"Precision: {row['mean_test_precision']}")
    print(f"Recall: {row['mean_test_recall']}")
    print(f"Specificity: {row['mean_test_specificity']}")
    print(f"F1 Score: {row['mean_test_f1']}")
    print("-" * 40)

參數組合: {'C': 3, 'gamma': 0.1, 'kernel': 'rbf'}
Accuracy: 0.7894752822074491
Precision: 0.8352920718437741
Recall: 0.7096013277706412
Specificity: 0.8659825280451642
F1 Score: 0.7573159498973333
----------------------------------------
參數組合: {'C': 2, 'gamma': 0.1, 'kernel': 'rbf'}
Accuracy: 0.790417408721581
Precision: 0.8376632339625265
Recall: 0.7080882603703661
Specificity: 0.8692758142396778
F1 Score: 0.7574887988495608
----------------------------------------
參數組合: {'C': 3, 'gamma': 0.2, 'kernel': 'rbf'}
Accuracy: 0.7743284799543211
Precision: 0.8110159158136119
Recall: 0.7024446667500313
Specificity: 0.843186076666632
F1 Score: 0.7429035735636382
----------------------------------------
參數組合: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
Accuracy: 0.7879256902810066
Precision: 0.8376972925935202
Recall: 0.70120556572351
Specificity: 0.8709894632918609
F1 Score: 0.7533610503393424
----------------------------------------
參數組合: {'C': 2, 'gamma': 0.2, 'kernel': 'rbf'}
Accuracy: 0.7797813940

In [12]:
print("最佳 Accuracy: %f 使用 %s" % (grid_result.cv_results_['mean_test_accuracy'][grid_result.best_index_], grid_result.best_params_))
print("最佳 Precision: %f 使用 %s" % (grid_result.cv_results_['mean_test_precision'][grid_result.best_index_], grid_result.best_params_))
print("最佳 Recall: %f 使用 %s" % (grid_result.cv_results_['mean_test_recall'][grid_result.best_index_], grid_result.best_params_))
print("最佳 Specificity: %f 使用 %s" % (grid_result.cv_results_['mean_test_specificity'][grid_result.best_index_], grid_result.best_params_))
print("最佳 F1: %f 使用 %s" % (grid_result.cv_results_['mean_test_f1'][grid_result.best_index_], grid_result.best_params_))

最佳 Accuracy: 0.789475 使用 {'C': 3, 'gamma': 0.1, 'kernel': 'rbf'}
最佳 Precision: 0.835292 使用 {'C': 3, 'gamma': 0.1, 'kernel': 'rbf'}
最佳 Recall: 0.709601 使用 {'C': 3, 'gamma': 0.1, 'kernel': 'rbf'}
最佳 Specificity: 0.865983 使用 {'C': 3, 'gamma': 0.1, 'kernel': 'rbf'}
最佳 F1: 0.757316 使用 {'C': 3, 'gamma': 0.1, 'kernel': 'rbf'}


In [14]:
result={'Accuracy': grid_result.cv_results_['mean_test_accuracy'][grid_result.best_index_],
    'Precision': grid_result.cv_results_['mean_test_precision'][grid_result.best_index_],
    'Recall': grid_result.cv_results_['mean_test_recall'][grid_result.best_index_],
    'Specificity': grid_result.cv_results_['mean_test_specificity'][grid_result.best_index_],
    'F1': grid_result.cv_results_['mean_test_f1'][grid_result.best_index_],
    'Best_Params': [{'C': 3, 'gamma': 0.1, 'kernel': 'rbf'}]}
df_result = pd.DataFrame([result])
df_result.to_csv('semantic_SVM_1.csv',index=False)